In [2]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', as_frame=True) # 숫자 데이터를 사용할 것임
import pandas as pd
mnist.frame.to_csv("num.csv", index=False)
targetDF=mnist.target
targetDF=targetDF.astype('int') # int로 타입 변경
targetDF.to_csv("ans.csv", index=False)

c:\Users\mathn\.conda\envs\Torch_PY38\lib\site-packages\sklearn\datasets\_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
featureDF=pd.read_csv("num.csv")/255 # MinMax 스케일러 적용
targetDF=pd.read_csv("ans.csv")["class"] # Series 형태로 뽑음 

In [4]:
x_train=featureDF.iloc[:60000]
y_train=targetDF.iloc[:60000]
x_test=featureDF.iloc[60000:]
y_test=targetDF.iloc[60000:]
print(f"x_train : {x_train.shape}, y_train : {y_train.shape}")
print(f"x_test : {x_test.shape}, y_test : {y_test.shape}")

x_train : (60000, 785), y_train : (60000,)
x_test : (10000, 785), y_test : (10000,)


In [5]:
# 학습용 & 검증용 분리 => 학습용 데이터셋에서 일부를 검증용으로 준비
# 검증용 : 학습 시 모델 최적화가 잘 되는지 검사하는 데이터셋
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3,stratify=y_train,random_state=5)
print(f"x_train : {x_train.shape}, y_train : {y_train.shape}")
print(f"x_val : {x_val.shape}, y_val : {y_val.shape}")

x_train : (42000, 785), y_train : (42000,)
x_val : (18000, 785), y_val : (18000,)


In [7]:
# 학습 진행
# 분류 방법 : Logistic Regression
from sklearn.linear_model import LogisticRegression

lr_model=LogisticRegression(max_iter=10000)
lr_model.fit(x_train,y_train)
train_score=lr_model.score(x_train,y_train)
test_score=lr_model.score(x_val, y_val)
print(f'train_score : {train_score}, test_score:{test_score}')

train_score : 0.9449285714285715, test_score:0.9195


In [8]:
from sklearn.linear_model import SGDClassifier
# Logistic Regression 알고리즘으로 학습 진행
lr_sgd=SGDClassifier(loss="log_loss", random_state=5)
lr_sgd.fit(x_train,y_train)
train_score=lr_sgd.score(x_train,y_train)
test_score=lr_sgd.score(x_val, y_val)
print(f'train_score : {train_score}, test_score:{test_score}')

train_score : 0.9193095238095238, test_score:0.9086666666666666


In [9]:
import imageio.v2 as imageio
import torch   
import torch.nn as nn              
import torch.nn.functional as F       
import torch.optim as optim     

def load_image(filename, start, end):
    tensor_list=[]
    for i in range(start,end+1):
        img_data=torch.tensor(imageio.imread(f"./img/{filename}{i}.png"))
        new_size = (100, 100)
        resized_img_data = F.interpolate(img_data.unsqueeze(0).permute(0, 3, 1, 2), size=new_size, mode='bilinear', align_corners=False)
        resized_img_data = resized_img_data.squeeze(0).permute(1, 2, 0)
        tensor_list.append(resized_img_data)
    return torch.stack(tensor_list)

sento=load_image("sento",0,19)
why=load_image("why",0,12)
x = torch.cat([sento, why], dim=0) # 사진 33개, shape : [33,100,100,4]
x=x.float()
y = [0 for _ in range(20)]+[1 for _ in range(13)] # 0 : 전투방, 1: 비밀방 

In [15]:
x=x.reshape(33,40000)

In [17]:
lr_sgd.fit(x,y)
train_score=lr_sgd.score(x,y)
print(f'train_score : {train_score}')

train_score : 1.0
